## S&P 500 Galaxy

Use this utility to update the returns and std_dev fields within investment-options.csv

Globals

In [1]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True
throttle_limit=100
wait_time=30

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# format price data
pd.options.display.float_format = '{:0.2f}'.format

In [4]:
# read in sp500.csv
sp500 = pd.read_csv('sp500.csv')
sp500.drop(columns=['Headquarters Location', 'Date added','CIK', 'Founded'], inplace=True)
sp500.rename(columns={'Security':'Description',
                      'GICS Sector':'Asset Class'}, inplace=True)
sp500.set_index("Symbol", inplace=True)
sp500

,Description,Asset Class,GICS Sub-Industry
Symbol,,,
MMM,3M,Industrials,Industrial Conglomerates
AOS,A. O. Smith,Industrials,Building Products
ABT,Abbott Laboratories,Health Care,Health Care Equipment
ABBV,AbbVie,Health Care,Biotechnology
ACN,Accenture,Information Technology,IT Consulting & Other Services
...,...,...,...
XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components
YUM,Yum! Brands,Consumer Discretionary,Restaurants
ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments


In [5]:
# read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Energy': 'US Stocks:Energy',
 'Materials': 'US Stocks:Materials',
 'Industrials': 'US Stocks:Industrials',
 'Consumer Discretionary': 'US Stocks:Consumer Discretionary',
 'Consumer Staples': 'US Stocks:Consumer Staples',
 'Health Care': 'US Stocks:Healthcare',
 'Financials': 'US Stocks:Financials',
 'Information Technology': 'US Stocks:Technology',
 'Communication Services': 'US Stocks:Communication Services',
 'Utilities': 'US Stocks:Utilities',
 'Real Estate': 'US Stocks:Real Estate'}

In [6]:
# Map sp500 GICS sectors to brownbear defined asset classes.
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

sp500['Asset Class'] = sp500.apply(_asset_class, axis=1)

# Yahoo finance uses '-' where '.' is used in symbol names.
sp500.index = sp500.index.str.replace('.', '-', regex=False)
sp500

,Description,Asset Class,GICS Sub-Industry
Symbol,,,
MMM,3M,US Stocks:Industrials,Industrial Conglomerates
AOS,A. O. Smith,US Stocks:Industrials,Building Products
ABT,Abbott Laboratories,US Stocks:Healthcare,Health Care Equipment
ABBV,AbbVie,US Stocks:Healthcare,Biotechnology
ACN,Accenture,US Stocks:Technology,IT Consulting & Other Services
...,...,...,...
XYL,Xylem Inc.,US Stocks:Industrials,Industrial Machinery & Supplies & Components
YUM,Yum! Brands,US Stocks:Consumer Discretionary,Restaurants
ZBRA,Zebra Technologies,US Stocks:Technology,Electronic Equipment & Instruments


In [7]:
# Make symbols list.
symbols = list(sp500.index)

In [8]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries, throttle_limit=throttle_limit, wait_time=wait_time)
bb.compile_timeseries(symbols)

MMM AOS ABT ABBV ACN ADBE AMD AES AFL A APD ABNB AKAM ALB ARE ALGN ALLE LNT ALL GOOGL GOOG MO AMZN AMCR AEE AEP AXP AIG AMT AWK AMP AME AMGN APH ADI ANSS AON APA APO AAPL AMAT APTV ACGL ADM ANET AJG AIZ T ATO ADSK ADP AZO AVB AVY AXON BKR BALL BAC BAX BDX BRK-B BBY TECH BIIB BLK BX BK BA BKNG BSX BMY AVGO BR BRO BF-B BLDR BG BXP CHRW CDNS CZR CPT CPB COF CAH KMX CCL CARR CAT CBOE CBRE CDW COR CNC CNP CF CRL SCHW CHTR CVX 
Throttle limit reached. Waiting for 30 seconds...
CMG CB CHD CI CINF CTAS CSCO C CFG CLX CME CMS KO CTSH CL CMCSA CAG COP ED STZ CEG COO CPRT GLW CPAY CTVA CSGP COST CTRA CRWD CCI CSX CMI CVS DHR DRI DVA DAY DECK DE DELL DAL DVN DXCM FANG DLR DFS DG DLTR D DPZ DASH DOV DOW DHI DTE DUK DD EMN ETN EBAY ECL EIX EW EA ELV EMR ENPH ETR EOG EPAM EQT EFX EQIX EQR ERIE ESS EL EG EVRG ES EXC EXE EXPE EXPD EXR XOM FFIV FDS FICO FAST FRT FDX FIS FITB FSLR FE FI F FTNT 
Throttle limit reached. Waiting for 30 seconds...
FTV FOXA FOX BEN FCX GRMN IT GE GEHC GEV GEN GNRC GD GIS GM G

In [9]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df = df[:]
df.tail()[['NVDA', 'WST', 'ADBE', 'NOW', 'AMD', 'PYPL', 'AAPL', 'MSFT', 'DHR', 'TMO', 'AMZN', 'DXCM', 'ODFL', 'REGN', 'ROL', 'FDX', 'TMUS', 'CDNS', 'CRM', 'MSCI']]

,NVDA,WST,ADBE,NOW,AMD,PYPL,AAPL,MSFT,DHR,TMO,AMZN,DXCM,ODFL,REGN,ROL,FDX,TMUS,CDNS,CRM,MSCI
Date,,,,,,,,,,,,,,,,,,,,
2025-04-28,108.73,215.91,368.62,937.41,96.39,64.93,210.14,391.16,195.91,421.34,187.70,71.18,149.00,610.86,56.02,211.50,236.77,285.74,265.64,534.95
2025-04-29,109.02,213.25,370.98,942.86,96.06,66.32,211.21,394.04,198.93,427.53,187.39,71.30,149.65,568.91,56.34,209.85,241.59,302.22,267.76,540.46
2025-04-30,108.92,211.29,374.98,955.01,97.35,65.84,212.50,395.26,199.33,429.00,184.42,71.38,153.28,598.76,57.13,210.33,246.95,297.74,268.71,545.11
2025-05-01,111.61,210.26,374.63,957.95,96.65,65.41,213.32,425.40,196.71,419.89,190.20,70.26,150.56,590.00,56.07,210.06,247.00,299.73,269.33,546.07
2025-05-02,114.50,211.29,380.87,977.36,98.80,67.24,205.35,435.28,199.05,423.55,189.98,81.62,158.01,605.61,56.67,219.72,248.88,308.15,275.08,555.82


In [10]:
# Calculate Annual Returns.
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns

MMM     50.57
AOS    -15.84
ABT     27.85
ABBV    26.46
ACN      3.27
        ...  
XYL     -3.87
YUM      7.66
ZBRA   -18.63
ZBH    -14.13
ZTS     -0.02
Length: 503, dtype: float64

In [11]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [12]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)

In [13]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)

In [14]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('ME').ffill()

In [15]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()

In [16]:
# Calculate 1 year, 3 year, and 5 year annualized standard deviation.
std_dev_1yr = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=1)
std_dev_3yr = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev_5yr = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=5)

In [17]:
# Read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]

In [18]:
# For each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = lines.copy()

# This is still slow (2.53 s).
for i, (index, row) in enumerate(sp500.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd_1yr = std_dev_1yr[symbol]*100
    sd_3yr = std_dev_3yr[symbol]*100
    sd_5yr = std_dev_5yr[symbol]*100

    out.append((
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}",'
        '"{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
    ).format(
        symbol, description, asset_class, ret_1mo, ret_3mo, ret_1yr, ret_3yr,
        ret_5yr, _vola, _ds_vola, sd_1yr, sd_3yr, sd_5yr
    ))

In [19]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')